In [1]:
from glob import glob
import pandas as pd
import os
import re

In [31]:
data_folder = "../data/thesis quantative results"
sampling_folders = glob(data_folder+"/*")
print(sampling_folders[:5])
print(f"We have {len(sampling_folders)} samples")

['../data/thesis quantative results\\2024-06-25 20-16-51', '../data/thesis quantative results\\2024-06-25 20-34-35', '../data/thesis quantative results\\2024-06-25 20-34-37', '../data/thesis quantative results\\2024-06-25 20-34-40', '../data/thesis quantative results\\2024-06-25 20-34-44']
We have 332 samples


In [32]:
sampling_num_interiors = []
sampling_num_CH = []
sampling_num_puzzles = []
sampling_total_num_points = []
sampling_ids = []

for folder in  sampling_folders:
    csv_paths = glob(os.path.join(folder,"*.csv"))
    count_puzzles = 0
    folder_puzzles_paths = []
    is_metdata_found = False

    for csv_path in csv_paths:
        csv_name = os.path.basename(csv_path)

        # Because the first kernel has always tiling, the metadata always starts with a letter
        # There should be only one
        if re.match("^[a-zA-Z]+.*",csv_name):
            is_metdata_found = True
            df_metadata = pd.read_csv(csv_path)
            counts_roles = df_metadata["role"].value_counts()
            try:
                sampling_num_interiors.append(counts_roles["interior"])
            except KeyError:
                sampling_num_interiors.append(0)
            
            sampling_num_CH.append(counts_roles["frame"])
        else:
            count_puzzles+=1
        
    if not is_metdata_found:
        print(f"Cannot find the sampling metadata csv of {folder}")

    sampling_num_puzzles.append(count_puzzles)
    containing_dir = os.path.basename(folder)
    sampling_ids.append(containing_dir)



df_all_samples = pd.DataFrame({
    "id": sampling_ids,
    "num_interior":sampling_num_interiors,
    "num_CH": sampling_num_CH,
    "num_puzzles":sampling_num_puzzles
})

df_all_samples["num_total_points"] = df_all_samples["num_interior"] + df_all_samples["num_CH"]

# A sampling must create a puzzle, remove all the sampling that were failed
df_all_samples.drop(df_all_samples[df_all_samples["num_puzzles"] == 0].index, inplace=True)
df_all_samples.head(50)

,id,num_interior,num_CH,num_puzzles,num_total_points
0,2024-06-25 20-16-51,5,5,6439,10
1,2024-06-25 20-34-35,2,5,61,7
2,2024-06-25 20-34-37,2,5,109,7
3,2024-06-25 20-34-40,1,6,155,7
4,2024-06-25 20-34-44,3,4,59,7
5,2024-06-25 20-34-46,2,6,471,8
6,2024-06-25 20-35-07,3,5,793,8
7,2024-06-25 20-35-45,4,4,579,8
8,2024-06-25 20-36-29,2,6,371,8
9,2024-06-25 20-39-07,3,7,24255,10


In [37]:
df_all_samples.groupby(["num_interior","num_CH"],as_index=False).agg({"num_puzzles":["mean","std","count"]})

num_interior num_CH   num_puzzles                   
                                mean          std count
0             0      3      1.000000          NaN     1
1             0      4      3.000000          NaN     1
2             0      5     11.000000          NaN     1
3             0      6     45.000000          NaN     1
4             0      7    178.000000    38.000000     4
5             1      3      1.000000          NaN     1
6             1      4      5.000000          NaN     1
7             1      5     36.000000    19.157244     3
8             1      6    128.866667    19.699562    45
9             1      7    664.000000   118.720961     4
10            1      8   3721.000000          NaN     1
11            2      4     17.000000          NaN     1
12            2      5     93.055556    19.982157   108
13            2      6    489.956522   123.836945    23
14            2      7   3046.000000   974.222620     8
15            2      8  16533.000000  5340.070412     2
16            3      3     15.000000          NaN     1
17            3      4     75.780488    30.400257    41
18            3      5    457.222222   159.945546    36
19            3      6   2241.888889   908.523589     9
20            3      7  16633.400000  5098.808174     5
21            4      3     69.000000          NaN     1
22            4      4    345.285714   124.854201     7
23            4      5   2184.142857   740.168321     7
24            4      6  15473.000000  3558.621756     6
25            5      4   1793.000000          NaN     1
26            5      5  10940.200000  4536.642733    10
27            6      4   8591.666667  5134.293070     3

In [24]:
df_all_samples[df_all_samples["num_CH"] == 3]

,id,num_interior,num_CH,num_puzzles,num_total_points
79,2024-06-26 00-34-28,4,3,131,7
318,CH-5-INT-2-14-28-21 to 3-3,3,3,17,6
322,i_0,0,3,83,3
328,sampled_4_puzzle1,1,3,11,4


In [11]:
df_all_samples.groupby(["num_interior","num_CH"]).size()

num_interior  num_CH
0             3           1
              4           1
              5           1
              6           1
              7           4
1             3           1
              4           1
              5           3
              6          45
              7           4
              8           1
2             4           1
              5         108
              6          23
              7           8
              8           2
3             3           1
              4          41
              5          36
              6           9
              7           5
4             3           1
              4           7
              5           7
              6           6
5             4           1
              5          10
6             4           3
dtype: int64